### Imports

In [ ]:
import pandas as pd 
import os
%matplotlib inline

from os import listdir, path
from utils import *
from tensorflow.keras.layers import Conv2D,Input,ZeroPadding2D,BatchNormalization,Flatten,Activation,Dense,MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers, losses
from sklearn.metrics import classification_report

### Loading Images

In [ ]:
image_dir="/home/pedrod33/Desktop/MRSI/ano1s2/AA/Brain_Tumor_Detection/archive/brain_tumor_dataset/"

### Making directory for augmented images

**A directory is formed using os.makedirs() function for augmented images(yes/ no). Note- custom directory is obtained in outputs folder.**

In [ ]:
if not path.exists('output/kaggle/working/augmented-images'):
    os.makedirs('output/kaggle/working/augmented-images')
if not path.exists('output/kaggle/working/augmented-images/yes'):
    os.makedirs('output/kaggle/working/augmented-images/yes')
if not path.exists('output/kaggle/working/augmented-images/no'):
    os.makedirs('output/kaggle/working/augmented-images/no')

## Augmentation of images 
**About the data:
The dataset contains 2 folders: yes and no which contains 253 Brain MRI Images. The folder yes contains 155 Brain MRI Images that are tumorous andno contains 98 Brain MRI Images that are non-tumorous.**

In [ ]:
IMG_SIZE = 224
X, y = read_images()
X = crop_images(X)
X = resize_and_rescale(X, IMG_SIZE)
X_train, y_train, X_val, y_val, X_test, y_test = split_and_shuffle(X, y)
#X_train, y_train = augment_data(X_train,y_train, IMG_SIZE=IMG_SIZE)

X_train = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_train])
X_val = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_val])
X_test = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_test])

## Visualization of data

In [ ]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))

In [ ]:
def build_model(input_shape, learn_rate=0.001):
    model = Sequential()
    model.add(Input(input_shape)) 
    model.add(Activation('leaky_relu')) 
    
    model.add(Conv2D(filters=64, kernel_size=4, input_shape=input_shape, strides = (1, 1)))
    model.add(MaxPooling2D((4,4), strides = (4,4),padding="valid"))
    model.add(Dropout(.3))
    
    model.add(Conv2D(filters=128, kernel_size=4, input_shape=input_shape, strides = (1, 1)))
    model.add(MaxPooling2D((4,4), strides = (4,4),padding="valid"))

    model.add(Flatten())
    model.add(Dense(units=512, activation="relu"))

    model.add(Dense(2, activation='sigmoid'))
    model.compile(optimizer=optimizers.Adam(learning_rate=learn_rate), loss=losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

    
    
    return model

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE,3)
model=build_model(IMG_SHAPE,0.001)
model.summary()

In [ ]:
results = model.fit(x=X_train, y=y_train, batch_size=32, epochs=22, validation_data=(X_val, y_val))

### Plotting of accuracy

### model.evaluate(X_test, y)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
plot_accuracy_comparison([results.history["accuracy"], results.history["val_accuracy"]],
                        "Training/Validation Accuracy Comparison",
                        ["Training Accuracy", "Validation Accuracy"])

In [ ]:
plot_accuracy_comparison([results.history["loss"], results.history["val_loss"]],
                        "Training/Validation Loss Comparison",
                        ["Training Accuracy", "Validation Accuracy"])

In [ ]:
y_pred=np.argmax(model.predict(X_test), axis=-1)
plot_confusion_matrix(y_test, y_pred)
print(classification_report(y_pred,y_test))

### Hyper-Parameter Selection

In [ ]:
learning_rates=[0.1, 0.01, 0.001, 0.0001, 0.00001]
epochs = 22
acc = []
loss = []
test = []
for lr in learning_rates:
    model = build_model(IMG_SHAPE, learn_rate=lr)
    results = model.fit(x=X_train, y=y_train, batch_size=32, epochs=22, validation_data=(X_val, y_val))
    print("Test with dataset and learning rate ",lr,":")
    model.evaluate(X_test, y_test)
    print()
    acc.append(results.history["val_accuracy"])
    loss.append(results.history["val_loss"])

In [ ]:
plot_accuracy_comparison(acc, "Learning Rate Comparison (Validation Accuracy)", learning_rates)

In [ ]:
plot_loss_comparison(loss, "Learning Rate Comparison (Validation Loss)", learning_rates)